In [2]:
import re, string, unicodedata
import nltk
from nltk.tokenize import TreebankWordTokenizer, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [3]:
file = open('case_example.txt', 'r').read()

In [5]:
# Step 1: Get rid of everything else, only leave the "Opinion" part

def opinion_doc(doc):
    start = re.search(r'\\\n\\pard\\pardeftab720\\ri0\\sb200\\qj\\partightenfactor0\n\n\\b \\cf3 Opinion', doc).start()
    stop = re.search(r'All Citations', doc).start()
    new_doc = doc[start:stop]
    return new_doc

a = opinion_doc(file)
a[:1000]

'\\\n\\pard\\pardeftab720\\ri0\\sb200\\qj\\partightenfactor0\n\n\\b \\cf3 Opinion\\\n\\pard\\pardeftab720\\ri0\\qj\\partightenfactor0\n\n\\b0 \\cf0 \\\n\\pard\\pardeftab720\\ri0\\sb200\\sa200\\qj\\partightenfactor0\n{\\field{\\*\\fldinst{HYPERLINK "http://www.westlaw.com/Link/Document/FullText?findType=h&pubNum=176284&cite=0249477901&originatingDoc=Ibdefc00c8b9e11d99dcc8cc3e68b51e9&refType=RQ&originationContext=document&vr=3.0&rs=cblt1.0&transitionType=DocumentItem&contextData=(sc.Search)"}}{\\fldrslt \\cf4 BOWMAN}}, Circuit Judge.\\\n\\pard\\pardeftab720\\ri0\\qj\\partightenfactor0\n\\cf0 \\\nIn this labor case, the National Labor Relations Board (Board) has petitioned to enforce its make-whole order against McKenzie Engineering Company and McKenzie Engineering has appealed from that order. This is the fourth time we have waded into this dispute between McKenzie Engineering, certain union carpenters, and the National Labor Relations Board. \n\\i See {\\field{\\*\\fldinst{HYPERLINK "ht

In [4]:
# Step 2. Extract all hyperlinks with http (still need to figure out how to only extract citations)

def find_url(doc): 
    url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', doc) 
    return url 

url_list = find_url(a)
print(len(url_list)) # length of the list of urls, which means there are 25 url links
print (url_list)

25
['http://www.westlaw.com/Link/Document/FullText?findType=h&pubNum=176284&cite=0249477901&originatingDoc=Ibdefc00c8b9e11d99dcc8cc3e68b51e9&refType=RQ&originationContext=document&vr=3.0&rs=cblt1.0&transitionType=DocumentItem&contextData=(sc.Search)', 'http://www.westlaw.com/Link/Document/FullText?findType=Y&serNum=2002578983&pubNum=506&originatingDoc=Ibdefc00c8b9e11d99dcc8cc3e68b51e9&refType=RP&originationContext=document&vr=3.0&rs=cblt1.0&transitionType=DocumentItem&contextData=(sc.Search)', 'http://www.westlaw.com/Link/Document/FullText?findType=Y&serNum=2000383996&pubNum=506&originatingDoc=Ibdefc00c8b9e11d99dcc8cc3e68b51e9&refType=RP&originationContext=document&vr=3.0&rs=cblt1.0&transitionType=DocumentItem&contextData=(sc.Search)', 'http://www.westlaw.com/Link/Document/FullText?findType=Y&serNum=1999154361&pubNum=506&originatingDoc=Ibdefc00c8b9e11d99dcc8cc3e68b51e9&refType=RP&originationContext=document&vr=3.0&rs=cblt1.0&transitionType=DocumentItem&contextData=(sc.Search)', 'http:/

In [7]:
# step 3: convert rtf to plain test

def striprtf(text):
   pattern = re.compile(r"\\([a-z]{1,32})(-?\d{1,10})?[ ]?|\\'([0-9a-f]{2})|\\([^a-z])|([{}])|[\r\n]+|(.)", re.I)
   # control words which specify a "destionation".
   destinations = frozenset((
      'aftncn','aftnsep','aftnsepc','annotation','atnauthor','atndate','atnicn','atnid',
      'atnparent','atnref','atntime','atrfend','atrfstart','author','background',
      'bkmkend','bkmkstart','blipuid','buptim','category','colorschememapping',
      'colortbl','comment','company','creatim','datafield','datastore','defchp','defpap',
      'do','doccomm','docvar','dptxbxtext','ebcend','ebcstart','factoidname','falt',
      'fchars','ffdeftext','ffentrymcr','ffexitmcr','ffformat','ffhelptext','ffl',
      'ffname','ffstattext','field','file','filetbl','fldinst','fldrslt','fldtype',
      'fname','fontemb','fontfile','fonttbl','footer','footerf','footerl','footerr',
      'footnote','formfield','ftncn','ftnsep','ftnsepc','g','generator','gridtbl',
      'header','headerf','headerl','headerr','hl','hlfr','hlinkbase','hlloc','hlsrc',
      'hsv','htmltag','info','keycode','keywords','latentstyles','lchars','levelnumbers',
      'leveltext','lfolevel','linkval','list','listlevel','listname','listoverride',
      'listoverridetable','listpicture','liststylename','listtable','listtext',
      'lsdlockedexcept','macc','maccPr','mailmerge','maln','malnScr','manager','margPr',
      'mbar','mbarPr','mbaseJc','mbegChr','mborderBox','mborderBoxPr','mbox','mboxPr',
      'mchr','mcount','mctrlPr','md','mdeg','mdegHide','mden','mdiff','mdPr','me',
      'mendChr','meqArr','meqArrPr','mf','mfName','mfPr','mfunc','mfuncPr','mgroupChr',
      'mgroupChrPr','mgrow','mhideBot','mhideLeft','mhideRight','mhideTop','mhtmltag',
      'mlim','mlimloc','mlimlow','mlimlowPr','mlimupp','mlimuppPr','mm','mmaddfieldname',
      'mmath','mmathPict','mmathPr','mmaxdist','mmc','mmcJc','mmconnectstr',
      'mmconnectstrdata','mmcPr','mmcs','mmdatasource','mmheadersource','mmmailsubject',
      'mmodso','mmodsofilter','mmodsofldmpdata','mmodsomappedname','mmodsoname',
      'mmodsorecipdata','mmodsosort','mmodsosrc','mmodsotable','mmodsoudl',
      'mmodsoudldata','mmodsouniquetag','mmPr','mmquery','mmr','mnary','mnaryPr',
      'mnoBreak','mnum','mobjDist','moMath','moMathPara','moMathParaPr','mopEmu',
      'mphant','mphantPr','mplcHide','mpos','mr','mrad','mradPr','mrPr','msepChr',
      'mshow','mshp','msPre','msPrePr','msSub','msSubPr','msSubSup','msSubSupPr','msSup',
      'msSupPr','mstrikeBLTR','mstrikeH','mstrikeTLBR','mstrikeV','msub','msubHide',
      'msup','msupHide','mtransp','mtype','mvertJc','mvfmf','mvfml','mvtof','mvtol',
      'mzeroAsc','mzeroDesc','mzeroWid','nesttableprops','nextfile','nonesttables',
      'objalias','objclass','objdata','object','objname','objsect','objtime','oldcprops',
      'oldpprops','oldsprops','oldtprops','oleclsid','operator','panose','password',
      'passwordhash','pgp','pgptbl','picprop','pict','pn','pnseclvl','pntext','pntxta',
      'pntxtb','printim','private','propname','protend','protstart','protusertbl','pxe',
      'result','revtbl','revtim','rsidtbl','rxe','shp','shpgrp','shpinst',
      'shppict','shprslt','shptxt','sn','sp','staticval','stylesheet','subject','sv',
      'svb','tc','template','themedata','title','txe','ud','upr','userprops',
      'wgrffmtfilter','windowcaption','writereservation','writereservhash','xe','xform',
      'xmlattrname','xmlattrvalue','xmlclose','xmlname','xmlnstbl',
      'xmlopen',
   ))
   # Translation of some special characters.
   specialchars = {
      'par': '\n',
      'sect': '\n\n',
      'page': '\n\n',
      'line': '\n',
      'tab': '\t',
      'emdash': '\u2014',
      'endash': '\u2013',
      'emspace': '\u2003',
      'enspace': '\u2002',
      'qmspace': '\u2005',
      'bullet': '\u2022',
      'lquote': '\u2018',
      'rquote': '\u2019',
      'ldblquote': '\201C',
      'rdblquote': '\u201D',
   }
   stack = []
   ignorable = False       # Whether this group (and all inside it) are "ignorable".
   ucskip = 1              # Number of ASCII characters to skip after a unicode character.
   curskip = 0             # Number of ASCII characters left to skip
   out = []                # Output buffer.
   for match in pattern.finditer(text):
      word,arg,hex,char,brace,tchar = match.groups()
      if brace:
         curskip = 0
         if brace == '{':
            # Push state
            stack.append((ucskip,ignorable))
         elif brace == '}':
            # Pop state
            ucskip,ignorable = stack.pop()
      elif char: # \x (not a letter)
         curskip = 0
         if char == '~':
            if not ignorable:
                out.append('\xA0')
         elif char in '{}\\':
            if not ignorable:
               out.append(char)
         elif char == '*':
            ignorable = True
      elif word: # \foo
         curskip = 0
         if word in destinations:
            ignorable = True
         elif ignorable:
            pass
         elif word in specialchars:
            out.append(specialchars[word])
         elif word == 'uc':
            ucskip = int(arg)
         elif word == 'u':
            c = int(arg)
            if c < 0: c += 0x10000
            if c > 127: out.append(chr(c)) #NOQA
            else: out.append(chr(c))
            curskip = ucskip
      elif hex: # \'xx
         if curskip > 0:
            curskip -= 1
         elif not ignorable:
            c = int(hex,16)
            if c > 127: out.append(chr(c)) #NOQA
            else: out.append(chr(c))
      elif tchar:
         if curskip > 0:
            curskip -= 1
         elif not ignorable:
            out.append(tchar)
   return ''.join(out)

b = striprtf(a)
b[:3000]

'Opinion, Circuit Judge.In this labor case, the National Labor Relations Board (Board) has petitioned to enforce its make-whole order against McKenzie Engineering Company and McKenzie Engineering has appealed from that order. This is the fourth time we have waded into this dispute between McKenzie Engineering, certain union carpenters, and the National Labor Relations Board. See  (denying petition to enforce Board\x92s order finding that McKenzie Engineering committed unfair labor practice by repudiating pre-hire agreement);  (reversing district court judgment entered in favor of carpenter union in their ERISA action for fringe-benefit contributions);  (granting enforcement of Board\x92s order finding that McKenzie committed unfair labor practices and remanding for determination of remedy). In this petition to enforce its make-whole order and McKenzie\x92s appeal from that order, the only issues are the extent of the back pay due to four discharged union carpenters (and their replaceme

In [6]:
# Step 2. Remove some other things
def remove_none_word(doc):
    new_doc = re.sub('\x92', "'", doc)
    new_doc = re.sub('\x93', '"', new_doc)
    new_doc = re.sub('\x94', '"', new_doc)
    return new_doc

c = remove_none_word(b)
c[:3000]

'Opinion, Circuit Judge.In this labor case, the National Labor Relations Board (Board) has petitioned to enforce its make-whole order against McKenzie Engineering Company and McKenzie Engineering has appealed from that order. This is the fourth time we have waded into this dispute between McKenzie Engineering, certain union carpenters, and the National Labor Relations Board. See  (denying petition to enforce Board\'s order finding that McKenzie Engineering committed unfair labor practice by repudiating pre-hire agreement);  (reversing district court judgment entered in favor of carpenter union in their ERISA action for fringe-benefit contributions);  (granting enforcement of Board\'s order finding that McKenzie committed unfair labor practices and remanding for determination of remedy). In this petition to enforce its make-whole order and McKenzie\'s appeal from that order, the only issues are the extent of the back pay due to four discharged union carpenters (and their replacements) a

In [7]:
file = open('processed_case_example1.txt', 'w')
file.write(c)

15813

In [8]:
# 3. Word Tokenization (after this step, we will be working on word level, not text level)

def word_tokenize(doc):
    doc = re.sub('[^\w]', ' ', doc)
    return TreebankWordTokenizer().tokenize(doc)

d = word_tokenize(c) # a list of words
print(d)

['Opinion', 'Circuit', 'Judge', 'In', 'this', 'labor', 'case', 'the', 'National', 'Labor', 'Relations', 'Board', 'Board', 'has', 'petitioned', 'to', 'enforce', 'its', 'make', 'whole', 'order', 'against', 'McKenzie', 'Engineering', 'Company', 'and', 'McKenzie', 'Engineering', 'has', 'appealed', 'from', 'that', 'order', 'This', 'is', 'the', 'fourth', 'time', 'we', 'have', 'waded', 'into', 'this', 'dispute', 'between', 'McKenzie', 'Engineering', 'certain', 'union', 'carpenters', 'and', 'the', 'National', 'Labor', 'Relations', 'Board', 'See', 'denying', 'petition', 'to', 'enforce', 'Board', 's', 'order', 'finding', 'that', 'McKenzie', 'Engineering', 'committed', 'unfair', 'labor', 'practice', 'by', 'repudiating', 'pre', 'hire', 'agreement', 'reversing', 'district', 'court', 'judgment', 'entered', 'in', 'favor', 'of', 'carpenter', 'union', 'in', 'their', 'ERISA', 'action', 'for', 'fringe', 'benefit', 'contributions', 'granting', 'enforcement', 'of', 'Board', 's', 'order', 'finding', 'that',

In [9]:
# sentence tokenizer

def sentence_tokenize(doc):
    return nltk.sent_tokenize(doc)

s = sentence_tokenize(c) # a list of words
print(s)

['Opinion, Circuit Judge.In this labor case, the National Labor Relations Board (Board) has petitioned to enforce its make-whole order against McKenzie Engineering Company and McKenzie Engineering has appealed from that order.', 'This is the fourth time we have waded into this dispute between McKenzie Engineering, certain union carpenters, and the National Labor Relations Board.', "See  (denying petition to enforce Board's order finding that McKenzie Engineering committed unfair labor practice by repudiating pre-hire agreement);  (reversing district court judgment entered in favor of carpenter union in their ERISA action for fringe-benefit contributions);  (granting enforcement of Board's order finding that McKenzie committed unfair labor practices and remanding for determination of remedy).", "In this petition to enforce its make-whole order and McKenzie's appeal from that order, the only issues are the extent of the back pay due to four discharged union carpenters (and their replacem

In [10]:
# 4. Normalization

# Step 1: Everything else

# Remove non-ASCII characters from list of tokenized words

def remove_non_ascii(words):
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)   
    return new_words

In [11]:
def lowercase(words):
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

In [12]:
def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub('[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

In [13]:
def remove_stopwords(words):
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

In [14]:
def normalize(words):
    words1 = remove_non_ascii(words)
    words2 = lowercase(words1)
    words3 = remove_punctuation(words2)
    words4 = remove_stopwords(words3)
    return words4

e = normalize(d)
print(e)

['opinion', 'circuit', 'judge', 'labor', 'case', 'national', 'labor', 'relations', 'board', 'board', 'petitioned', 'enforce', 'make', 'whole', 'order', 'mckenzie', 'engineering', 'company', 'mckenzie', 'engineering', 'appealed', 'order', 'fourth', 'time', 'waded', 'dispute', 'mckenzie', 'engineering', 'certain', 'union', 'carpenters', 'national', 'labor', 'relations', 'board', 'see', 'denying', 'petition', 'enforce', 'board', 'order', 'finding', 'mckenzie', 'engineering', 'committed', 'unfair', 'labor', 'practice', 'repudiating', 'pre', 'hire', 'agreement', 'reversing', 'district', 'court', 'judgment', 'entered', 'favor', 'carpenter', 'union', 'erisa', 'action', 'fringe', 'benefit', 'contributions', 'granting', 'enforcement', 'board', 'order', 'finding', 'mckenzie', 'committed', 'unfair', 'labor', 'practices', 'remanding', 'determination', 'remedy', 'petition', 'enforce', 'make', 'whole', 'order', 'mckenzie', 'appeal', 'order', 'issues', 'extent', 'back', 'pay', 'due', 'four', 'dischar

In [15]:
# Step 2. Stemming 

def my_stemmer(words):  
    stem_list = []
    for word in words:
        stem_str = LancasterStemmer().stem(word)
        stem_list.append(stem_str)
    return stem_list


# Step 3: Lemmatization for verbs

def my_lemma(words):
    lemma_list = []
    for word in words:
        lemma_str = WordNetLemmatizer().lemmatize(word, pos='v')
        lemma_list.append(lemma_str)
    return lemma_list

# combine two functions into one. One gives stemmed tokens, the other gives lemmatized tokens. Depends on our goal

def stem_and_lemmatize(words):
    stems = my_stemmer(words)
    lemmas = my_lemma(words)
    return stems, lemmas

stems, lemmas = stem_and_lemmatize(d)
print('Stemmed:\n', stems)
print('\nLemmatized:\n', lemmas)

Stemmed:
 ['opin', 'circuit', 'judg', 'in', 'thi', 'lab', 'cas', 'the', 'nat', 'lab', 'rel', 'board', 'board', 'has', 'petit', 'to', 'enforc', 'it', 'mak', 'whol', 'ord', 'against', 'mckenzie', 'engin', 'company', 'and', 'mckenzie', 'engin', 'has', 'ap', 'from', 'that', 'ord', 'thi', 'is', 'the', 'four', 'tim', 'we', 'hav', 'wad', 'into', 'thi', 'disput', 'between', 'mckenzie', 'engin', 'certain', 'un', 'carp', 'and', 'the', 'nat', 'lab', 'rel', 'board', 'see', 'deny', 'petit', 'to', 'enforc', 'board', 's', 'ord', 'find', 'that', 'mckenzie', 'engin', 'commit', 'unfair', 'lab', 'pract', 'by', 'repudy', 'pre', 'hir', 'agr', 'revers', 'district', 'court', 'judg', 'ent', 'in', 'fav', 'of', 'carp', 'un', 'in', 'their', 'eris', 'act', 'for', 'fring', 'benefit', 'contribut', 'grant', 'enforc', 'of', 'board', 's', 'ord', 'find', 'that', 'mckenzie', 'commit', 'unfair', 'lab', 'pract', 'and', 'remand', 'for', 'determin', 'of', 'remedy', 'in', 'thi', 'petit', 'to', 'enforc', 'it', 'mak', 'whol', 